In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

df = pd.read_csv("mobiles.csv")
df.head(2)

,screen_size,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales
0,Very Small,64,2,1,1,1800,4.5,38645,32999,0.17,127.52
1,Small,64,4,2,1,2815,4.5,244,57149,0.04,1.39


# Q1

In [65]:
df_q1 = df.copy()
sales_mean = df_q1["sales"].mean()
# sales_stddev = np.sqrt(df_q1["sales"].std()) # 그냥 std 가 표준편차 맞았음 ㄷㄷ;
sales_stddev = df_q1["sales"].std()
q1 = sales_mean - (2*sales_stddev)
q3 = sales_mean + (2*sales_stddev)
# df_q1 = df_q1[(df_q1["sales"] >= q1) & (df_q1["sales"] <= q3)] # 2표준편차보다 크다고 했으니 이것도 틀렸다. ;; 작은건 포함이다. 또한 이상치만 계산하는게 문제다!! 난 이상치를 뺐다 ㅋ
df_q1 = df_q1[df_q1["sales"] > q3]
df_q1["cam_cnt"] = df_q1["num_rear_camera"] + df_q1["num_front_camera"]
df_q1["kpi"] = df_q1["ROM"] / 32 + df_q1["RAM"] / 2 + df_q1["cam_cnt"] + df_q1["battery_capacity"] / 1000

round(df_q1["kpi"].mean(), 2)



11.01

In [64]:
# 답

stat_mean = df["sales"].mean()
stat_std = df["sales"].std()
stat_out = stat_mean + 2*stat_std
stat_out #1 146.5515

df_q1 = df.loc[df["sales"] > stat_out, ].reset_index(drop = True) # 이상치라고 볼 수 있다.
df_q1

df_q1["idx"] = (df_q1["ROM"] / 32) + (df_q1["RAM"] / 2) + \
                (df_q1["num_front_camera"] + df_q1["num_rear_camera"]) + \
                (df_q1["battery_capacity"] / 1000)
round(df_q1["idx"].mean(), 2)


11.01

# Q2

In [28]:
df_q2 = df[["num_rear_camera", "sales", "battery_capacity", "ratings", "num_of_ratings", "sales_price", "discount_percent"]]
df_q2 = df_q2[df_q2["num_rear_camera"].astype("int") != 1]
df_q2.iloc[:,1:].corr()
# num_of_ratings 0.949114
round(0.949114, 2)

0.95

In [78]:
# 답
df_q2 = df.loc[df["num_rear_camera"] != 1, "battery_capacity":].copy()
df_q2.corr().abs().round(2)["sales"][:-1].max() 
df_q2.corr().replace(1, np.nan).max() # 1이 없다는 가정하에 쓸 수 있다.
df_q2.corr().reset_index()
df_q2.corr().reset_index().melt(id_vars="index")
df_q2_melt = df_q2.corr().reset_index().melt(id_vars="index")
df_q2_melt = df_q2_melt.loc[df_q2_melt["index"] != df_q2_melt["variable"], ]
df_q2_melt.pivot(index="index", columns="variable", values="value")


battery_capacity    0.03
ratings             0.23
num_of_ratings      0.95
sales_price         0.25
discount_percent    0.22
sales               1.00
Name: sales, dtype: float64

# Q3


In [58]:
df_q3 = df.copy()
df_q3 = pd.concat([df_q3[["sales"]], df_q3.drop(columns=["sales"])], axis=1)

X = df_q3.iloc[:, 1:]
Y = df_q3.iloc[:, 0]

objects = []
for key in df_q3.dtypes.to_dict():
    if df_q3.dtypes.to_dict()[key] == 'object':
        objects.append(key)

for index in objects:
    df_q3 = pd.concat([df_q3.drop(columns=[index]), pd.get_dummies(df_q3[index])], axis=1)



train, test = train_test_split(df_q3, test_size=0.2, random_state=123)
len(train), len(test)

scaler = MinMaxScaler()
train_norm = scaler.fit_transform(train)
test_norm = scaler.transform(test)

max = 0
max_rmse = 0
for k in [3,5,7,9,11]:
    model = KNeighborsRegressor(k)
    model.fit(X=train_norm[:,1:], y=train_norm[:,0])
    y_pred = model.predict(X=test_norm[:, 1:])
    rmse = np.sqrt(mean_squared_error(y_true=test_norm[:,0], y_pred=y_pred))
    print(k, rmse)
    if rmse > max_rmse:
        max = k

k  ## 틀렸다!! rmse 가 가장 작은게 가장 좋은거다!! ㅠㅠㅠㅠ


3 0.08186677375964535
5 0.09879109824384892
7 0.107669855645971
9 0.11232111394853059
11 0.1136902366621185


11

In [93]:
#답
df_q3_dum = pd.get_dummies(df, dtype="int") #최신 버전만 가능 -> 알아서 명목형 변경
# df_q3_dum = pd.get_dummies(df, columns=["screen_size"]) # 시험버전

df_q3_dum = df_q3_dum.set_index("sales").reset_index() # 앞으로 옮기는 쉬운 방법

df_train, df_test = train_test_split(df_q3_dum, train_size=0.8, random_state=123)
model_nor = MinMaxScaler().fit(df_train)
arr_train_nor = model_nor.transform(df_train)
arr_test_nor = model_nor.transform(df_test)

ls_k = [3,5,7,9,11]
ls_rmse = []
for k in ls_k:
  model_knn = KNeighborsRegressor(n_neighbors=k)
  model_knn.fit(X=arr_train_nor[:, 1:], y=arr_train_nor[:,0])
  pred = model_knn.predict(arr_test_nor[:, 1:])
  val_rmse = mean_squared_error(y_true=arr_test_nor[:,0], y_pred= pred) ** 0.5 
  ls_rmse = ls_rmse + [val_rmse]

ser_rmse = pd.Series(ls_rmse, index=ls_k)
ser_rmse


,sales,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
13,5.90,256,4,2,1,2815,4.6,745,79149,0.02,0,1,0,0,0
266,0.18,32,2,2,1,5000,4.0,213,8238,0.01,1,0,0,0,0
330,1.01,256,12,3,1,4500,4.5,110,91999,0.20,1,0,0,0,0
220,28.41,128,6,3,1,4310,4.3,14204,19999,0.09,0,1,0,0,0
240,28.92,64,4,3,1,5000,4.5,28927,9999,0.09,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,15.27,64,4,3,1,5000,4.3,9853,15499,0.03,1,0,0,0,0
114,3.54,32,2,1,1,5000,4.3,4850,7299,0.08,1,0,0,0,0
31,1.52,128,4,2,1,2815,4.5,244,62149,0.04,0,0,1,0,0
188,4.89,128,8,4,2,4200,4.3,1746,27999,0.06,1,0,0,0,0


# Q3 추가문항

Q3. 추가 지시사항 \
다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가? \
※ 정규화 되지 않은 값으로 반올림하여 소수점 첫째 자리까지 출력하시오 \
※ KNN 모델을 사용하며 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하시오.


ROM: 256 \
RAM: 6 \
num_rear_camera: 4 \
num_front_camera: 1 \
battery_capacity: 4000 \
ratings: 4.3 \
num_of_ratings: 25000 \
sales_price: 85000 \
discount_percent: 0.05 \
screen_size: "Large"

In [100]:

model = KNeighborsRegressor(3)
#df_t1 = pd.DataFrame(dict(ROM=123 ...))
df_t1 = df_test.head(1).reset_index(drop=True)
df_t1["ROM"] = 256
df_t1["RAM"] = 6
df_t1["num_rear_camera"] = 4
df_t1["num_front_camera"] = 1
df_t1["battery_capacity"] = 4000
df_t1["ratings"] = 4.3
df_t1["num_of_ratings"] = 25000
df_t1["sales_price"] = 85000
df_t1["discount_percent"] = 0.05
df_t1["screen_size_Large"] = 1
df_t1["screen_size_Medium"] = 0
df_t1["screen_size_Small"] = 0
df_t1["screen_size_Very Large"] = 0
df_t1["screen_size_Very Small"] = 0

# MinMaxScaler().fit_transform(df_t1)  #땡
arr_t1_nor = model_nor.transform(df_t1)
model_knn_best = model.fit(X=arr_train_nor[:,1:], y=arr_train_nor[:, 0]) # 다시 트레인 시켜야 한다. 아까 11에서 끝났으니
pred_t1 = model_knn_best.predict(arr_t1_nor[:, 1:])
pred_t1 # 여기서 끝나면 안된다.판매지수를 원래 값으로 뽑아야 한다!! 으악

arr_t1_nor[0,0] = pred_t1 # 그냥 역변환 하면 안되고, 반드시 원래 shape와 동일하게 만들어야 한다.
arr_t1_inv = model_nor.inverse_transform(arr_t1_nor)
arr_t1_inv

df_t1_inv = pd.DataFrame(arr_t1_inv, columns=df_t1.columns)
df_t1_inv


array([[0.0119438 , 0.49206349, 0.45454545, 1.        , 0.        ,
        0.42307692, 0.625     , 0.05308122, 0.51815842, 0.09302326,
        1.        , 0.        , 0.        , 0.        , 0.        ]])